### Grober Imageloader 

In [1]:
# import the necessary packages
import cv2
class SimplePreprocessor:
	def __init__(self, width, height, inter=cv2.INTER_AREA):
		# store the target image width, height, and interpolation
		# method used when resizing
		self.width = width
		self.height = height
		self.inter = inter
	def preprocess(self, image):
		# resize the image to a fixed size, igndfdoring the aspect
		# ratio
		return cv2.resize(image, (self.width, self.height),
			interpolation=self.inter)

In [1]:
import numpy as np
import cv2
import os
class SimpleDatasetLoader:
    def __init__(self, preprocessors=None):
        # store the image preprocessor
        self.preprocessors = preprocessors
        # if the preprocessors are None, initialize them as an
        # empty list
        if self.preprocessors is None:
            self.preprocessors = []

    def load(self, imagePaths, verbose=-1):
            # initialize the list of features and labels
            data = []
            labels = []
            # loop over the input images
            for (i, imagePath) in enumerate(imagePaths):
                # load the image and extract the class label assuming
                # that our path has the following format:
                # /path/to/dataset/{class}/{image}.jpg
                image = cv2.imread(imagePath)
                label = imagePath.split(os.path.sep)[-2]

                # check to see if our preprocessors are not None
                if self.preprocessors is not None:
                    # loop over the preprocessors and apply each to
                    # the image
                    for p in self.preprocessors:
                        image = p.preprocess(image)
                # treat our processed image as a "feature vector"
                # by updating the data list followed by the labels
                data.append(image)
                labels.append(label)
                # show an update every `verbose` images
                if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
                    print("[INFO] processed {}/{}".format(i + 1,
                        len(imagePaths)))
            # return a tuple of the data and labels
            return (np.array(data), np.array(labels))

## Experimete Bild --> Featureextraction

Ecken finden mit CornerHarris

In [2]:
import numpy as np
import cv2 as cv
filename = 'kz1.jpeg'
img = cv.imread(filename)
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
gray = np.float32(gray)
dst = cv.cornerHarris(gray,2,3,0.1)
#result is dilated for marking the corners, not important
dst = cv.dilate(dst,None)
# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.01*dst.max()]=[0,0,255]
cv.namedWindow('dst',cv.WINDOW_NORMAL)
cv.imshow('dst',img)
if cv.waitKey(0) & 0xff == 27:
    cv.destroyAllWindows()

Kontur finden mit cv2.threshold und cv2.findContur <br>
Numeric data für Tabelle: Relation Größe/Breite

In [1]:
import cv2
import numpy as np
import imutils
from scipy.spatial import distance

# Load image, grayscale, Gaussian blur, threshold
image = cv2.imread('weiss.JPG')
undraw_imgage = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (9,9), 0)
temp_thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY_INV)[1]

# Konturen + Länge/ Größen-Verhältnis
height, width = temp_thresh.shape
oben = 0.1
unten = 0.1
cnts = cv2.findContours(temp_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
c_real = max(cnts, key=cv2.contourArea)
th2 = 120

#Verschiedene Thresholds anschauen für cv2.threshold (schwarz/weiß Bild)
for th1 in range(50,140,1):
    oben = 0.1
    unten = 0.1
    rechts = 0.1
    links = 0.1
    temp_thresh = cv2.threshold(blur, th1, 255, cv2.THRESH_BINARY_INV)[1]

    # Erste Hauptkontur finden (größte)
    cnts = cv2.findContours(temp_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(cnts[0])==0:
        continue
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    c = max(cnts, key=cv2.contourArea)

    #Parameter für: Checken ob Hauptkontur mittig im Bild. Zählt Konturpunkte ob obenuntenrechtslinks
    for cc in c:
        if cc[0][1]>(height/2):
            oben +=1
        else:
            unten+=1
        if cc[0][0]>(width/2):
            rechts +=1
        else:
            links +=1
    #Zweite Kontur finden (zweitgröße)
    max_area = 0
    c2 = c
    for cn in cnts:
        if cv2.contourArea(cn)<cv2.contourArea(c) and cv2.contourArea(cn)>max_area:
            max_area = cv2.contourArea(cn)
            c2 = cn


#Mein Kack Algorithmus für ein optimierte Konturfindung
#Motto: Kontur groß = gut ; Kontur mittig (grob) = gut
    if cv2.contourArea(c_real)<cv2.contourArea(c):
        if 0.3 < abs(oben/unten) and abs(oben/unten) < 3 and 0.3 < abs(links/rechts) and abs(links/rechts) < 3:
            print("Servus")
            th2 = th1
            c_mini = c2
            c_real = c
        # Else: Vielleicht Zweitkontur besser?
        else:
            oben = 0.1
            unten = 0.1
            for cc in c2:
                if cc[0][1]>(height/2):
                    oben +=1
                else:
                    unten+=1
                if cc[0][0]>(width/2):
                    rechts +=1
                else:
                    links +=1
            if 0.3 < abs(oben/unten) and abs(oben/unten) < 3 and 0.3 < abs(links/rechts) and abs(links/rechts) < 3 and cv2.contourArea(c_real)<cv2.contourArea(c2):
                print("Servus2")
                th2 = th1
                c_mini = c
                c_real = c2



thresh = cv2.threshold(blur, th2, 255, cv2.THRESH_BINARY_INV)[1]
# Obtain outer coordinates
left = tuple(c_real[c_real[:, :, 0].argmin()][0])
right = tuple(c_real[c_real[:, :, 0].argmax()][0])
top = tuple(c_real[c_real[:, :, 1].argmin()][0])
bottom = tuple(c_real[c_real[:, :, 1].argmax()][0])

# Draw dots onto image
cv2.drawContours(image, [c_mini], -1, (36, 135, 12), 2)
cv2.drawContours(image, [c_real], -1, (36, 255, 12), 2)
cv2.circle(image, left, 8, (0, 50, 255), -1)
cv2.circle(image, right, 8, (0, 255, 255), -1)
cv2.circle(image, top, 8, (255, 50, 0), -1)
cv2.circle(image, bottom, 8, (255, 255, 0), -1)

#Distance und Relation
d_breit = distance.euclidean(left, right)
d_groß = distance.euclidean(top, bottom)

real = abs(d_breit/d_groß)

#Ganz viele prints der Ausgaben
print('left: {}'.format(left))
print('right: {}'.format(right))
print('top: {}'.format(top))
print('bottom: {}'.format(bottom))
print("--------------------")
print("Größe: ",d_groß)
print("Breite: ",d_breit)
print("Real Größe: ",real)
cv2.namedWindow("thresh", cv2.WINDOW_NORMAL) 
cv2.namedWindow("image", cv2.WINDOW_FREERATIO) 
cv2.imshow('thresh', thresh)
cv2.imshow('image', image)
cv2.waitKey()
cv2.destroyAllWindows()

Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus2
Servus
Servus
Servus
Servus
Servus
Servus
Servus
Servus
left: (0, 0)
right: (2655, 3983)
top: (0, 0)
bottom: (0, 3983)
--------------------
Größe:  3983.0
Breite:  4786.785351360556
Real Größe:  1.2018040048607974


Erste Versuche zu Checken wo mehr Konturen sind

In [51]:
#--------- Konturen Relation ObenUntenRechtsLinks -----------

relimg1 = undraw_imgage.copy()

# Mittelpunkts von maxima nehmen
mittelp = ( left[0]+int((right[0]-left[0])/2),      top[1] + int((bottom[1] - top[1])/2)   )
c_num = c_real

#Vom Mittelpunkt schauen wo mehr Konturpunkte liegen
rel_oben = 0.1
rel_unten = 0.1
rel_links = 0.1
rel_rechts = 0.1
for cc in c_num:
    cv2.circle(relimg1, cc[0], 2, (0, 50, 255), -1)
    if cc[0][1]>mittelp[1]:
        rel_oben +=1
    else:
        rel_unten+=1
    if cc[0][0]>mittelp[0]:
        rel_rechts +=1
    else:
        rel_links+=1

rel_obenunten = abs(rel_oben/rel_unten)
rel_rechtslinks = abs(rel_rechts/rel_links)
allkon1 = oben+unten
print("Alle Konturen:", allkon1)
print("ObenuntenREL: ",rel_obenunten)
print("RechtsLinksREL: ",rel_rechtslinks)

cv2.circle(relimg1, mittelp, 10, (100, 50, 255), -1)
cv2.namedWindow('relimage', cv2.WINDOW_NORMAL) 
cv2.imshow('relimage', relimg1)
cv2.waitKey()
cv2.destroyAllWindows()


Alle Konturen: 3255.2
ObenuntenREL:  1.0153035198095561
RechtsLinksREL:  0.5057807140134671


Eckendetector - cv2.goodFeaturesToTrack

In [53]:
cornimg = gray.copy()
cornimg2 = blur.copy()


corners = cv2.goodFeaturesToTrack(cornimg,1000,0.1,2)
corners2 = cv2.goodFeaturesToTrack(thresh,1000,0.1,2)



corners = np.int0(corners)
corners2 = np.int0(corners2)

for corner in corners:
    x,y = corner.ravel()
    cv2.circle(cornimg,(x,y),2,(255,0,0),-1)

for corner2 in corners2:
    x,y = corner2.ravel()
    cv2.circle(cornimg2,(x,y),2,(255,0,0),-1)

cv2.namedWindow("corners", cv2.WINDOW_NORMAL) 
cv2.imshow("corners",cornimg)

cv2.namedWindow("corners2", cv2.WINDOW_NORMAL) 
cv2.imshow("corners2",cornimg2)
cv2.waitKey()
cv2.destroyAllWindows()



Blob Detector - In Bearbeitung - wenig Hoffnung, wäre cool wenns geht<br>
Quelle: https://learnopencv.com/blob-detection-using-opencv-python-c/

In [46]:


Blobimg = gray.copy()
Img = cv2.imread('kz1.jpeg', cv2.IMREAD_GRAYSCALE)
params = cv2.SimpleBlobDetector_Params()
# Set Area filtering parameters
params.filterByArea = False
params.minArea = 50

# Set Circularity filtering parameters
params.filterByCircularity = False
params.minCircularity = 0.01

# Set Convexity filtering parameters
params.filterByConvexity = False
params.minConvexity = 0.01
	
# Set inertia filtering parameters
params.filterByInertia = False
params.minInertiaRatio = 0.01
detector = cv2.SimpleBlobDetector_create(params)
keypoints = detector.detect(Img)
blank = np.zeros((1, 1))
im_with_keypoints = cv2.drawKeypoints(Blobimg, keypoints, blank, (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.namedWindow("Blob", cv2.WINDOW_NORMAL) 
cv2.imshow("Blob", im_with_keypoints)
cv2.waitKey()
cv2.destroyAllWindows()

import PyQt5
import matplotlib.pyplot as plt
%matplotlib qt
imgplot = plt.imshow(im_with_keypoints)

Test Canny Edge detector

In [2]:
#weiss.JPG cv.getTrackbarPos('nlm_thresh:', source_window)
cannyimg = cv2.imread('weiss.JPG')
cannygray = cv2.cvtColor(cannyimg, cv2.COLOR_BGR2GRAY)
cannyblur = cv2.GaussianBlur(cannygray, (9,9), 0)
global threshold10
global threshold20
threshold10 = 120
threshold20 = 120
maxScaleUp = 255
scaleFactor = 1
windowName = "canny"
trackbarValue1 = "Threshold1"
trackbarValue2 = "Threshold2"
cv2.namedWindow(windowName, cv2.WINDOW_NORMAL)

def flexthresh(*args):
    global threshold10
    # Get the scale factor from the trackbar
    threshold10=args[0]
    # Resize the image
    edges = cv2.Canny(image=cannyblur, threshold1=threshold10,threshold2=threshold20)
    cv2.imshow(windowName, edges)

def flexthresh2(*args):
    global threshold20
    # Get the scale factor from the trackbar
    threshold20 = args[0]
    # Resize the image
    edges = cv2.Canny(image=cannyblur, threshold1=threshold10,threshold2=threshold20)
    cv2.imshow(windowName, edges)

cv2.createTrackbar(trackbarValue1, windowName, scaleFactor, maxScaleUp, flexthresh)
cv2.createTrackbar(trackbarValue2, windowName, scaleFactor, maxScaleUp, flexthresh2)


cv2.imshow(windowName, cannyblur)
cv2.waitKey()
cv2.destroyAllWindows()